In [1]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
import pandas as pd

In [2]:
#Loading Data
df = pd.read_excel('SourceData2022.xlsx')
df

,ID,tweet,label
0,3,The lie that coronavirus came from a bat or a ...,Fake
1,4,The health experts had predicted the virus cou...,Fake
2,8,The Centers for Disease Control and Prevention...,Fake
3,10,Warm weather will kill coronavirus. U.S. Presi...,Unverified
4,15,Using a hair dryer to breathe in hot air can c...,Fake
...,...,...,...
4124,4197,“[The coronavirus is] “new” yet it was lab-cre...,Fake
4125,4199,A post claims that you have to wear a disposab...,Fake
4126,4202,Clickbait / Russian scientists have a cure for...,Fake
4127,4203,Coronavirus outbreak linked to eating bats,Fake


In [3]:
def populate_features(strtweet):
    Covidval=0
    CDCval=0
    centerdiseasecontrolval=0
    vaccineval=0
    paymentval=0
    if 'COVID-19' in strtweet :
        Covidval =1
    if 'CDC' in strtweet :
        CDCval =1   
    if 'center disease control' in strtweet :
        centerdiseasecontrolval =1   
    if 'vaccine' in strtweet :
        vaccineval =1   
    if 'payment' in strtweet :
        paymentval =1     
    if Covidval==1 or CDCval==1 or centerdiseasecontrolval==1 or vaccineval==1 or paymentval==1:
        dr = { 'COVID-19': Covidval,  'CDC':CDCval, 'center disease control':centerdiseasecontrolval,
              'vaccine':vaccineval,'payment':paymentval}
        return dr

In [4]:
#Onehot encoding of labels
one_hot = pd.get_dummies(df['label'])
df = df.join(one_hot)
#del df["ID"]

In [5]:
df

,ID,tweet,label,Disinformative,Fake,MisInformative,Real,Unverified
0,3,The lie that coronavirus came from a bat or a ...,Fake,0,1,0,0,0
1,4,The health experts had predicted the virus cou...,Fake,0,1,0,0,0
2,8,The Centers for Disease Control and Prevention...,Fake,0,1,0,0,0
3,10,Warm weather will kill coronavirus. U.S. Presi...,Unverified,0,0,0,0,1
4,15,Using a hair dryer to breathe in hot air can c...,Fake,0,1,0,0,0
...,...,...,...,...,...,...,...,...
4124,4197,“[The coronavirus is] “new” yet it was lab-cre...,Fake,0,1,0,0,0
4125,4199,A post claims that you have to wear a disposab...,Fake,0,1,0,0,0
4126,4202,Clickbait / Russian scientists have a cure for...,Fake,0,1,0,0,0
4127,4203,Coronavirus outbreak linked to eating bats,Fake,0,1,0,0,0


In [6]:
#Creating empty Dataframe
column_names = ["ID","COVID-19", "CDC", "center disease control","vaccine","payment"]
dfInherited = pd.DataFrame(columns = column_names)
dfInherited

,ID,COVID-19,CDC,center disease control,vaccine,payment


In [7]:
#Tranforming input to binary format
for row in df.itertuples(index=True, name='Pandas'):
    Covidval=0
    CDCval=0
    centerdiseasecontrolval=0
    vaccineval=0
    paymentval=0
    if 'COVID-19' in row.tweet :
        Covidval =1
    if 'CDC' in row.tweet :
        CDCval =1   
    if 'center disease control' in row.tweet :
        centerdiseasecontrolval =1   
    if 'vaccine' in row.tweet :
        vaccineval =1   
    if 'payment' in row.tweet :
        paymentval =1     
    if Covidval==1 or CDCval==1 or centerdiseasecontrolval==1 or vaccineval==1 or paymentval==1:
        dr = {'ID':row.ID, 'COVID-19': Covidval,  'CDC':CDCval, 'center disease control':centerdiseasecontrolval,
              'vaccine':vaccineval,'payment':paymentval,'Disinformative':row.Disinformative,'Fake':row.Fake,
              'MisInformative':row.MisInformative,'Real':row.Real,'Unverified':row.Unverified}
        dfInherited = dfInherited.append(dr, ignore_index = True)
del dfInherited["ID"]
dfInherited


,COVID-19,CDC,center disease control,vaccine,payment,Disinformative,Fake,MisInformative,Real,Unverified
0,1,1,0,0,0,0.0,1.0,0.0,0.0,0.0
1,1,0,0,0,0,0.0,1.0,0.0,0.0,0.0
2,1,0,0,0,0,0.0,0.0,0.0,1.0,0.0
3,1,0,0,0,0,0.0,1.0,0.0,0.0,0.0
4,1,0,0,1,0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
764,0,0,0,1,0,0.0,1.0,0.0,0.0,0.0
765,0,0,0,1,0,0.0,1.0,0.0,0.0,0.0
766,1,0,0,0,0,0.0,1.0,0.0,0.0,0.0
767,0,0,0,1,0,0.0,1.0,0.0,0.0,0.0


In [8]:
#Converting floating point labels into integer
dfInherited = dfInherited.astype(int)
dfInherited

,COVID-19,CDC,center disease control,vaccine,payment,Disinformative,Fake,MisInformative,Real,Unverified
0,1,1,0,0,0,0,1,0,0,0
1,1,0,0,0,0,0,1,0,0,0
2,1,0,0,0,0,0,0,0,1,0
3,1,0,0,0,0,0,1,0,0,0
4,1,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
764,0,0,0,1,0,0,1,0,0,0
765,0,0,0,1,0,0,1,0,0,0
766,1,0,0,0,0,0,1,0,0,0
767,0,0,0,1,0,0,1,0,0,0


In [9]:
#Converting to array
dataset=dfInherited.to_numpy()
dataset

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 1, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [10]:
# split into input (X) and output (y) variables
X = dataset[:,0:5]
y = dataset[:,5:]
print(X)
print(y)

[[1 1 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 ...
 [1 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 1 0]]
[[0 1 0 0 0]
 [0 1 0 0 0]
 [0 0 0 1 0]
 ...
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]]


In [11]:
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=5, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(5, activation='softmax'))


In [12]:
# compile the keras model
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


In [13]:
# fit the keras model on the dataset
model.fit(X, y, epochs=150, batch_size=10, verbose=0)


In [14]:
# evaluate the keras model
_, accuracy = model.evaluate(X, y, verbose=0)
print('Accuracy: %.2f' % (accuracy*100))

Accuracy: 83.36


In [15]:
#Testing an value
testDR=populate_features("The spread of COVID-19 is leveling off")
testDR

{'COVID-19': 1,
 'CDC': 0,
 'center disease control': 0,
 'vaccine': 0,
 'payment': 0}

In [16]:
#Converting to dataframe
testdf = pd.DataFrame(testDR, index=[0])
testdf

,COVID-19,CDC,center disease control,vaccine,payment
0,1,0,0,0,0


In [17]:
#Convet to array
testdataset=testdf.to_numpy()
testdataset

array([[1, 0, 0, 0, 0]], dtype=int64)

In [18]:
#Predicting
predictions =model.predict(testdataset)
rounded = [round(x) for x in predictions[0]]
predictions

array([[0.00344769, 0.85689604, 0.00194053, 0.11086648, 0.02684925]],
      dtype=float32)

In [19]:
#final prediction
rounded

[0, 1, 0, 0, 0]